In [ ]:
!pip install diffusers transformers torch torchvision matplotlib numpy reportlab

In [ ]:
pip install pillow opencv-python numpy

In [10]:
from diffusers import DiffusionPipeline
import torch
import gc

# Free up GPU memory before loading the model
torch.cuda.empty_cache()
gc.collect()

# Load Comic Diffusion Model with Optimized Settings
model_id = "ogkalu/Comic-Diffusion"

pipe = DiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,  # Use FP16 for lower memory
    safety_checker=None,  # Disable safety checker for performance (Use responsibly)
).to("cuda")

# Optimize for Low VRAM
pipe.enable_attention_slicing()
pipe.enable_sequential_cpu_offload()

print(" Model loaded successfully!")

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

An error occurred while trying to fetch C:\Users\amitm\.cache\huggingface\hub\models--ogkalu--Comic-Diffusion\snapshots\ff684f581ab24e094e2055d9422e9ee076d139a8\unet: Error no file named diffusion_pytorch_model.safetensors found in directory C:\Users\amitm\.cache\huggingface\hub\models--ogkalu--Comic-Diffusion\snapshots\ff684f581ab24e094e2055d9422e9ee076d139a8\unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch C:\Users\amitm\.cache\huggingface\hub\models--ogkalu--Comic-Diffusion\snapshots\ff684f581ab24e094e2055d9422e9ee076d139a8\vae: Error no file named diffusion_pytorch_model.safetensors found in directory C:\Users\amitm\.cache\huggingface\hub\models--ogkalu--Comic-Diffusion\snapshots\ff684f581ab24e094e2055d9422e9ee076d139a8\vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.

 Model loaded successfully!


In [11]:
#  Get user input
movie_description = input("🎬 Enter a 25+ word movie description: ")

🎬 Enter a 25+ word movie description:  Attack on Titan follows Eren Yeager, a young man living in a world where humanity is on the brink of extinction due to giant humanoid creatures known as Titans that devour humans. After his hometown is destroyed and his mother is killed by Titans, Eren joins the military with his friends Mikasa Ackerman and Armin Arlert. As they uncover dark secrets about the Titans, they discover that their world is enclosed by massive walls to protect them, and Eren possesses the mysterious ability to transform into a Titan himself. As the story unfolds, the trio uncovers the truth about the Titans' origins, humanity's true history, and the political struggles within their own society. Eren's journey leads to intense battles, moral dilemmas, and the revelation of a global conspiracy. Ultimately, the conflict reaches a boiling point, where Eren and his allies must face the consequences of their actions and choices in a war that will determine the future of humani

In [8]:
import time

# Generate Poster
start_time = time.time()

poster = pipe(
    movie_description, 
    width=768,  # Higher quality resolution
    height=1024, 
    num_inference_steps=50  # High-quality generation
).images[0]

# Save & Display Poster
poster_path = "movie_poster.png"
poster.save(poster_path)
poster.show()

print(f"Movie Poster generated in {time.time() - start_time:.2f} sec")

Token indices sequence length is longer than the specified maximum sequence length for this model (94 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['gallant blacksmith in love with elizabeth allies with sparrow in pursuit of the pirates .']


  0%|          | 0/50 [00:00<?, ?it/s]

Movie Poster generated in 164.10 sec


In [ ]:
from gpt4all import GPT4All

# Download & Load the Model (Replace with a valid model name)
gpt = GPT4All("mistral-7b-openorca.Q4_0.gguf", allow_download=True)

# Verify installation
print("GPT4All Model Loaded Successfully!")

In [9]:
from gpt4all import GPT4All
from PIL import Image, ImageDraw, ImageFont

# Load GPT4All Model
gpt = GPT4All("mistral-7b-openorca.Q4_0.gguf")

# Generate a Movie Title & Tagline
prompt = f"Generate a compelling movie title and tagline for this plot: {movie_description}\n\nTitle: "
response = gpt.generate(prompt)

# Extract Title & Tagline
lines = response.strip().split("\n")
movie_title = lines[0].replace("Title:", "").strip() if len(lines) > 0 else "Unknown Title"
tagline = lines[1].replace("Tagline:", "").strip() if len(lines) > 1 else "Unknown Tagline"

print(f"🎬Generated Movie Title: {movie_title}")
print(f"🎬Generated Tagline: {tagline}")

# Load the Generated Movie Poster
poster_path = "movie_poster.png"  # Path to the generated poster
poster = Image.open(poster_path)

# Convert Image to Editable Format
draw = ImageDraw.Draw(poster)
width, height = poster.size

# Load Font (Uses default if arial.ttf not found)
try:
    title_font = ImageFont.truetype("arial.ttf", 80)  # Large font for title
    tagline_font = ImageFont.truetype("arial.ttf", 40)  # Smaller font for tagline
except:
    title_font = ImageFont.load_default()
    tagline_font = ImageFont.load_default()

# Define Text Positions (Center Align)
title_x = width // 2
title_y = int(height * 0.1)  # Title at top

tagline_x = width // 2
tagline_y = int(height * 0.85)  # Tagline at bottom

# Add AI-Generated Title & Tagline
draw.text((title_x, title_y), movie_title, font=title_font, fill="white", anchor="mm")
draw.text((tagline_x, tagline_y), tagline, font=tagline_font, fill="white", anchor="mm")

# Save Final Poster with AI-Generated Text
output_path = "final_movie_poster.png"
poster.save(output_path)

print(f"Movie Poster saved as {output_path}")
poster.show()  # Display final poster

🎬Generated Movie Title: "Curse of the Undead Pirate"
🎬Generated Tagline: "The Caribbean's Most Wanted - Captain Jack Sparrow and a Cursed Crew!"
Movie Poster saved as final_movie_poster.png


In [ ]:
import torch
print("PyTorch Version:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())
print("CUDA Device Count:", torch.cuda.device_count())
